In [90]:
#Imports

#Data
import pandas as pd
import numpy as np


#Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

#ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import  TfidfTransformer
from sklearn.pipeline import Pipeline

from sklearn.datasets import make_regression
from sklearn.metrics import confusion_matrix,classification_report

from sklearn.preprocessing import OneHotEncoder

df=pd.read_csv('data/xyz_corp_lending_data.csv',sep='\t')
df=df[df['desc'].isna()==False][['desc','purpose']]
df[['purpose']] = df[['purpose']].astype('category').apply(lambda x: x.cat.codes)
df=df[(df['purpose']==2)|(df['purpose']==1)]
df['purpose']=df['purpose'].replace(2,0)
#df['desc']=df['desc'].apply(lambda x: x.split('>')[1][:-4] if x[-4:]=='<br>' else x)


C:\Users\mmoy\AppData\Local\Temp\ipykernel_23132\2017195591.py:27: DtypeWarning: Columns (17,45,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('data/xyz_corp_lending_data.csv',sep='\t')


In [93]:
def get_model_nlp(method='initial'):
    X=df['desc']
    y=df['purpose']
    if method=='initial':
        cv = CountVectorizer()
        X = cv.fit_transform(X)
        #undersample = RandomUnderSampler(sampling_strategy='majority')
        #X,y = undersample.fit_resample(X, y)
        X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)
        model_nlp = MultinomialNB()
        model_nlp.fit(X_train,y_train)
        predictions = model_nlp.predict(X_test)
    else:
        model_nlp = Pipeline([
        ('bow', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('classifier', MultinomialNB()),]) 
        X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)
        model_nlp.fit(X_train,y_train)
        predictions = model_nlp.predict(X_test) 
    print(confusion_matrix(y_test,predictions))
    print('\n')
    print(classification_report(y_test,predictions))
    return model_nlp
model_nlp=get_model_nlp(method='initial')


[[18208  2679]
 [ 4776  2907]]


              precision    recall  f1-score   support

           0       0.79      0.87      0.83     20887
           1       0.52      0.38      0.44      7683

    accuracy                           0.74     28570
   macro avg       0.66      0.63      0.63     28570
weighted avg       0.72      0.74      0.72     28570

